In [1]:
from system import *
s1 = system('o-x-o',MMA=True,ManyVariables=False,TwoPhotonResonance= True)

/home/yorgos/.local/lib/python3.8/site-packages/qutip/__init__.py:91: UserWarning: Old version of Cython detected: needed 0.29.20, got 0.29.14.
  warnings.warn(


Initializing system o-x-o  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblad operators ...

System  o-x-o  initialized in 30.1 seconds.


# Effective Hamiltonian as Real part

Will output $Q_n$ from expression:

$$\Delta_n = - \dfrac{\Omega^2}{\gamma} \text{Re}\{Q_n \} $$

In [2]:
diag = 0  #input n corresponding to diag element to view

var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g','gamma0','De0','g0',domain='positive')
var('DEt',domain='positive',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='positive',  latex_name =r'\tilde{\Delta }_e')
var('De0t',domain='positive',  latex_name =r'\tilde{\Delta }_{e0}')
var('dct',domain='positive',  latex_name =r'\tilde{\delta }_c')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c',domain='positive',  latex_name =r'c')


#Take only one of the compl conj terms, choose the one with the shortest length when shortned
min_expr_len = 10**10
for term in range(2):
    b = s1.eff_hamiltonian_gs[diag,diag]
    #b = b.subs(gamma0=gamma)
    
    b = b.operands()[1].operands()[term]  #  + b.operands()[1].operands()[3] use if obtained with sage


    b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    b = b.subs(De=+0.5*I*gamma   +Det*gamma)
    
    b = b.subs(De0= +0.5*I*gamma0 +De0t*gamma)
    
    #var('a0', latex_name =r'a_0')
    #b = b.subs(De0=+0.5*I*gamma   +Det*gamma*a0)
    #var('n')
    b = b.subs(DE=DEg*gamma)
    b = b.subs(De=Deg*gamma)
    var('a0', latex_name =r'a_0')
    b = b.subs(De0=a0*Deg*gamma)
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))
    
    var('R0',domain='positive',  latex_name =r'R_0')
    b = b.subs(g0=g*sqrt(R0))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)
    
    var('r0',domain='real',latex_name =r'r_0')
    b = b.subs(gamma0=gamma*r0)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b*gamma  #Omega has to be substituted with Omega/gamma
    

    b = b._mathematica_().Factor()._sage_()
    b = MMA_simplify(b,full= False)
    #holds the smallest expression
    if len(str(b))<min_expr_len:
        min_expr_len = len(str(b))
        h_eff = b
print('Element is the real part of this expression times  Ω^2/γ')
show( (symround(h_eff,digits=1)))

Element is the real part of this expression times  Ω^2/γ
symround: Deleted coefficient 1.1e-16


(c^2 + 0.33*c + 0.021)/(-0.66*I*C*R_f*(c + 0.25)^2 + (c^2 + 0.33*c + 0.021)*DEt)

## Show the result separated in Numerator and denominator multiplying by a factor

In [60]:
factor = 1#*16*4


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(h_eff.numerator().expand()),full=True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(h_eff.denominator().expand()),full= not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


-2*Det*c*dct - 0.500*I*Det*dct^2 + C*c + 0.500*I*C*dct



 Latex
-2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} - 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} + C {c} + 0.500i \, C {\tilde{\delta }_c}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


(Det*c + 0.500*I*Det*dct - 0.500*I*C)*C*R_f - (2*Det*c*dct + 0.500*I*Det*dct^2 - C*c - 0.500*I*C*dct)*DEt



 Latex
{\left({\tilde{\Delta }_e} {c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c} - 0.500i \, C\right)} C R_{f} - {\left(2 \, {\tilde{\Delta }_e} {c} {\tilde{\delta }_c} + 0.500i \, {\tilde{\Delta }_e} {\tilde{\delta }_c}^{2} - C {c} - 0.500i \, C {\tilde{\delta }_c}\right)} {\tilde{\Delta }_E}


# Lindblad Operators

Choose which of the Lindblad operators to view and which non zero element of it

In [30]:

def lind(lind_op,which):    
    L_matrix_orig = s1.lindblad_list[lind_op]
    L_matrix      = s1.eff_lindblad_list[lind_op] #lindblad_list[lind_op] ##(gamma_g=0,gamma_f=gamma)

    L_nonzeros = []
    L_nonzeros_pos = []
    #find non zero elements
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))

    #show what these elements do 
    #for t in L_nonzeros_pos:
    #    print(f' |{s1.gs_e1_dec_states[t[0]]}> < {s1.gs_e1_dec_states[t[1]]} | ')

    L_elem = L_nonzeros[which]

    var('gamma_f')
    var('R0',domain='real',latex_name =r'R_0')
    L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
    L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
    L_elem = L_elem.subs(De0=+0.5*I*gamma0   +De0t*gamma*R0)
    #L_elem = L_elem.subs(DE=gamma*DEg)
    #L_elem = L_elem.subs(De=gamma*Deg)                     
    #COUPLINGS
    b = L_elem
    #ratio  (g_0/g)^2
    b = b.subs(g0=g*sqrt(R0))
    
    var('R_f',domain='real')#ratio  (g_f/g)^2
    b = b.subs(g_f=g*sqrt(R_f))

    var('R_v',domain='real',  latex_name =r'R_{\nu}') #ratio (v/g)^2
    b = b.subs(v=g*sqrt(R_v))

    var('r_b',domain='real')
    b = b.subs(kappa_b=gamma/r_b)


    var('r_g',domain='real',latex_name =r'r_g')
    b = b.subs(gamma_g=gamma*r_g)


    var('r_f',domain='real',latex_name =r'r_f')
    b = b.subs(gamma_f=gamma*r_f)


    b = b.subs(g = sqrt(C*kappa_c*gamma)) 
    
    b = b.subs(r_b = c/(C*R_v))
    
    b = b.subs(dc = (dct+0.5*I)*kappa_c )
    
    
    L_elem = b
    
  
    L_elem = L_elem._mathematica_().PowerExpand()._sage_()
    L_elem = L_elem._mathematica_().Factor()._sage_()


    L_elem = MMA_simplify(L_elem,full=not True)

    L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))

    

    show( symround(L_elem))

    print(f' |{s1.gs_e1_dec_states[L_nonzeros_pos[which][0]]}> < {s1.gs_e1_dec_states[L_nonzeros_pos[which][1]]} | \n')
    print(f' Original coefficient :')
    show(s1.L_coeffs[lind_op])
    return L_elem


## View Lindblad results

In [37]:
lind_op = 6  #which lindblad operator
which   = 1  #which non zero element to access
L_elem = lind(lind_op,which)

print(latex(symround(L_elem)))


((4*c^2 + 3*c + 0.25)*C*(De0t + Det) - (-6*I*c^2 - 2*I*c - 0.12*I)*De0t*Det - C^2*(4*I*c + 0.50*I))*Omega*sqrt(r_g)/(((4*De0t*Det*(c + 0.25)^2 - (2*I*De0t*c + 2*I*Det*c + 0.50*I*De0t + 0.50*I*Det)*C - C^2)*C*R_f + ((4*c^2 + 3*c + 0.25)*C*(De0t + Det) - (-6*I*c^2 - 2*I*c - 0.12*I)*De0t*Det - C^2*(4*I*c + 0.50*I))*DEt)*sqrt(gamma))

 |0000g001> < 0000g001 | 

 Original coefficient :


sqrt(gamma_g)

\frac{{\left({\left(4 \, {c}^{2} + 3 \, {c} + 0.25\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} - {\left(-6 i \, {c}^{2} - 2 i \, {c} - 0.12i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} - C^{2} {\left(4 i \, {c} + 0.50i\right)}\right)} {\Omega} \sqrt{{r_g}}}{{\left({\left(4 \, {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} {\left({c} + 0.25\right)}^{2} - {\left(2 i \, {\tilde{\Delta }_{e0}} {c} + 2 i \, {\tilde{\Delta }_e} {c} + 0.50i \, {\tilde{\Delta }_{e0}} + 0.50i \, {\tilde{\Delta }_e}\right)} C - C^{2}\right)} C R_{f} + {\left({\left(4 \, {c}^{2} + 3 \, {c} + 0.25\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} - {\left(-6 i \, {c}^{2} - 2 i \, {c} - 0.12i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} - C^{2} {\left(4 i \, {c} + 0.50i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}}


In [40]:
factor = -8


print('numerator')
nu = symround(SR(str(MMA_simplify(factor*(L_elem.numerator().expand()),full=not True)).replace('Sqrt','sqrt'))\
              ,digits=3)
show(nu)
print("\n\n Latex")
print(latex(nu))

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print('\n\ndenominator')
den = symround(SR(str(MMA_simplify(factor*(L_elem.denominator().expand()),full=not True)).replace('Sqrt','sqrt'))\
               ,digits=3)
show(den)
print("\n\n Latex")
print(latex(den))

numerator


(2*(16*c^2 + 12*c + 1)*C*(De0t + Det) + (48*I*c^2 + 16*I*c + I)*De0t*Det + C^2*(-32*I*c - 4*I))*Omega*sqrt(r_g)



 Latex
{\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} + {\left(48 i \, {c}^{2} + 16 i \, {c} + i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} + C^{2} {\left(-32 i \, {c} - 4 i\right)}\right)} {\Omega} \sqrt{{r_g}}
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


denominator


((32*De0t*Det*(c + 0.250)^2 + (-16*I*De0t*c - 16*I*Det*c - 4*I*De0t - 4*I*Det)*C - 8*C^2)*C*R_f + (2*(16*c^2 + 12*c + 1)*C*(De0t + Det) + (48*I*c^2 + 16*I*c + I)*De0t*Det + C^2*(-32*I*c - 4*I))*DEt)*sqrt(gamma)



 Latex
{\left({\left(32 \, {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} {\left({c} + 0.250\right)}^{2} + {\left(-16 i \, {\tilde{\Delta }_{e0}} {c} - 16 i \, {\tilde{\Delta }_e} {c} - 4 i \, {\tilde{\Delta }_{e0}} - 4 i \, {\tilde{\Delta }_e}\right)} C - 8 \, C^{2}\right)} C R_{f} + {\left(2 \, {\left(16 \, {c}^{2} + 12 \, {c} + 1\right)} C {\left({\tilde{\Delta }_{e0}} + {\tilde{\Delta }_e}\right)} + {\left(48 i \, {c}^{2} + 16 i \, {c} + i\right)} {\tilde{\Delta }_{e0}} {\tilde{\Delta }_e} + C^{2} {\left(-32 i \, {c} - 4 i\right)}\right)} {\tilde{\Delta }_E}\right)} \sqrt{{\gamma}}


# Expand linearly in the limit:   $C\rightarrow \infty$ routine

In [ ]:
#with mathematica
q = b.subs(C=1/x)
q = MMA_simplify(q)
tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
tayl = tayl.subs(x=1/C)

show(symround(tayl))


((2*I*R_v*c_k + 0.66*I)*R_v*c_k + 0.042*I)/(((2*I*DE + 1.3*R_f)*R_v^2*c_k^2 + (0.66*I*DE + R_f)*R_v*c_k + 0.042*I*DE + 0.083*R_f)*C)